# Torch Tensors 
### Notes and code based on youtube videos by user Patrick Loeber: https://www.youtube.com/playlist?list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4

In [3]:
import torch
import numpy as np

In [4]:
x = torch.randint(20, size=(2,2))
y = torch.randint(6, size=(2,2))

print('x = ', x, 'y = ', y)

x =  tensor([[18,  0],
        [15,  0]]) y =  tensor([[0, 1],
        [0, 5]])


Element-wise operations written as $\textit{x+y, x-y, x*y, x/y}$ or $\textit{x.add(y)}$. Add a trailing underscore to do the operation in place. 

In [ ]:
print(x + y)
y.add_(x)
print(y)
if y.all() == (x+y).all():
    print("Same thing.")

tensor([[54,  1],
        [45,  5]])
tensor([[54,  1],
        [45,  5]])
True


Can use .item() to extract (scalar) tensor component as long as the tensor is sliced properly.

In [16]:
z = torch.randint(10, (2, 2))
print(z)
scalar = z[0, 1]
print(scalar, scalar.item())
print(type(scalar), type(scalar.item()))

tensor([[6, 3],
        [4, 8]])
tensor(3) 3
<class 'torch.Tensor'> <class 'int'>


$\textit{t.view()}$ can be used to reshape a tensor. Reshape dimensions must have the same product as previous dimension product ie $m \times n$ maps to $a \times b$ where $mn = ab$.

In [ ]:
a = torch.rand(4, 4)
print(a)
b = a.view(-1, 2) #reshape -1 makes automatic choice of other dimension when given one dimension as
#next input
print(b)

tensor([[0.8957, 0.7276, 0.0832, 0.1964],
        [0.1976, 0.3439, 0.6051, 0.2549],
        [0.7726, 0.7531, 0.9875, 0.0601],
        [0.2903, 0.4330, 0.2080, 0.5390]])
tensor([[0.8957, 0.7276],
        [0.0832, 0.1964],
        [0.1976, 0.3439],
        [0.6051, 0.2549],
        [0.7726, 0.7531],
        [0.9875, 0.0601],
        [0.2903, 0.4330],
        [0.2080, 0.5390]])


Can turn a numpy array into a torch tensor and vice versa. Care needs to be taken though as the tensor and array both occupy the same memory location if only working with the CPU (not GPU) so changes to one are also changes to the other.

In [ ]:
o = torch.ones(5)
print(o)
p = o.numpy() 
print(type(p)) 

s = np.ones(5)
print(s)
t = torch.from_numpy(s)
print(type(t))

tensor([1., 1., 1., 1., 1.])
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1.]
<class 'torch.Tensor'>


When initialising a tensor, setting the $\textit{requires\_grad}$ equal to True let's torch know that you will want to take the gradient of this tensor later on. This attribute needs to be specified before back propagation.

In [21]:
x = torch.randn(3, requires_grad=True) 
print(x)

y = x + 2
print(y)

tensor([-0.9104, -0.2441, -0.3120], requires_grad=True)
tensor([1.0896, 1.7559, 1.6880], grad_fn=<AddBackward0>)


Since requires_grad is True, torch tracks the operations made in the grad_fn attribute of the tensors and creates a computation graph for back propagation later. The forward pass calculates the output $y = f(x)$ and since gradient is specified, torch automatically creates a function for us which is used in back propagation to calculate the gradient. $y$ has attribute grad_fn which points to gradient function dy/dx.